In [107]:
import pandas as pd
df = pd.read_csv(r'../datasets/imdb_indonesian_movies_2.csv')

In [108]:
df.head()

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [109]:
df.drop('judul_film', axis=1, inplace=True)

### Ubah Label menjadi angka menggunakan OneHotEnconding--->dummies

In [110]:
category = pd.get_dummies(df['genre'])
df = pd.concat([df, category], axis=1)
df.drop(columns='genre', inplace=True)

In [111]:
df.head()

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0


##### mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [112]:
sinopsis = df['ringkasan_sinopsis'].values
label = df.iloc[:,1:].values

##### bagi data untuk training dan data untuk testing.

In [113]:
from sklearn.model_selection import train_test_split

sinopsis_train, sinopsis_test, label_train, label_test = train_test_split(sinopsis, label, test_size=0.2)

#### ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer

In [114]:
cek = df['ringkasan_sinopsis'].str.split().apply(len).tolist()
cek = pd.DataFrame(cek, columns=['words_count'])

In [115]:
cek.max()

words_count    1729
dtype: int64

In [116]:
cek.sum()*0.2

words_count    22630.8
dtype: float64

In [117]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=20000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_train)
# tokenizer.fit_on_texts(sinopsis_test)

### Mengubah token ke sequences

In [118]:
sinopsis_train_seq = tokenizer.texts_to_sequences(sinopsis_train)
sinopsis_test_seq = tokenizer.texts_to_sequences(sinopsis_test)

In [119]:
padded_train = pad_sequences(sinopsis_train_seq)
padded_test = pad_sequences(sinopsis_test_seq)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. Jangan lupa panggil fungsi compile dan tentukan optimizer serta loss function yang akan dipakai oleh model.

In [120]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=20000, output_dim=140),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

2023-04-03 05:17:24.507738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 05:17:24.519115: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 05:17:24.534884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [121]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [122]:
history = model.fit(padded_train, label_train, epochs=30,
                    validation_data=(padded_test, label_test),
                    verbose=2)

Epoch 1/30


2023-04-03 05:17:27.100383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 05:17:27.104093: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 05:17:27.107152: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

26/26 - 51s - loss: 1.6102 - accuracy: 0.2114 - val_loss: 1.6096 - val_accuracy: 0.1791 - 51s/epoch - 2s/step
Epoch 2/30
26/26 - 47s - loss: 1.5576 - accuracy: 0.2948 - val_loss: 1.5941 - val_accuracy: 0.2886 - 47s/epoch - 2s/step
Epoch 3/30
26/26 - 42s - loss: 1.0674 - accuracy: 0.6381 - val_loss: 1.6441 - val_accuracy: 0.3483 - 42s/epoch - 2s/step
Epoch 4/30
26/26 - 31s - loss: 0.4055 - accuracy: 0.8818 - val_loss: 2.4778 - val_accuracy: 0.3632 - 31s/epoch - 1s/step
Epoch 5/30
26/26 - 29s - loss: 0.0853 - accuracy: 0.9826 - val_loss: 2.6945 - val_accuracy: 0.3134 - 29s/epoch - 1s/step
Epoch 6/30
26/26 - 30s - loss: 0.0212 - accuracy: 0.9975 - val_loss: 3.1246 - val_accuracy: 0.3483 - 30s/epoch - 1s/step
Epoch 7/30
26/26 - 33s - loss: 0.0089 - accuracy: 1.0000 - val_loss: 3.2036 - val_accuracy: 0.3383 - 33s/epoch - 1s/step
Epoch 8/30
26/26 - 31s - loss: 0.0025 - accuracy: 1.0000 - val_loss: 3.4870 - val_accuracy: 0.3483 - 31s/epoch - 1s/step
Epoch 9/30
26/26 - 30s - loss: 0.0013 - acc

KeyboardInterrupt: 